In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

import warnings 
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
sample = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [4]:
#This is my first effort, using a simple, densely connected feed forward model. It attained a score of 0.97014. 

# Y_train = train["label"]
# X_train = train.drop(labels = ["label"],axis = 1) 

# X_train = X_train / 255.0
# test = test / 255.0

# x_train, y_train, x_val, y_val = train_test_split(X_train, Y_train, test_size = 0.15)

# epochs = 50
# learning_rate = 0.01
# decay_rate = learning_rate / epochs
# momentum = 0.8

# model = keras.models.Sequential()
# model.add(keras.layers.Dense(300, activation = 'LeakyReLU'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(keras.layers.Dense(200, activation = 'LeakyReLU'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(keras.layers.Dense(100, activation = 'LeakyReLU'))
# model.add(keras.layers.Dense(10, activation = 'softmax'))

# opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
# model.compile(loss = 'sparse_categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

# fit = model.fit(x_train, x_val, batch_size = 128, 
#                     epochs = 240, validation_data =(y_train, y_val))

# pred = model.predict(test)
# pred_class = np.argmax(pred, axis=-1)
# out = sample.copy()
# out['Label'] = pred_class
# out.to_csv("submission.csv", index=False)

In [5]:
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1) 

X_train = X_train / 255.0
test = test / 255.0

X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

x_train, y_train, x_val, y_val = train_test_split(X_train, Y_train, test_size = 0.15)

In [6]:
# data_augmentation = keras.Sequential(
#     [layers.RandomRotation(0.05),
#      layers.RandomZoom(0.10),
#      layers.RandomTranslation(height_factor=(-0.1, 0.1),
#                               width_factor=(-0.1, 0.1),
#                               fill_mode='nearest')])

In [7]:
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
#                                             patience=3, 
#                                             verbose=1, 
#                                             factor=0.5, 
#                                             min_lr=0.00001)

In [8]:
epochs = 50
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.8
batch_size = 64

In [9]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [10]:
inputs = keras.Input(shape=(28,28,1))
# x = data_augmentation(inputs)
x = layers.Conv2D(filters=128, kernel_size=5, activation = 'relu')(inputs)
x = BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation = 'relu')(x)
x = Dropout(rate = 0.25)(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation = 'relu')(x)
x = Dropout(rate = 0.25)(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = Dense(256, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dropout(rate = 0.5)(x)

outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

opt = keras.optimizers.RMSprop(learning_rate=learning_rate, momentum=momentum, decay=decay_rate)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

2022-06-28 05:27:05.965117: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
fit = model.fit_generator(datagen.flow(x_train,x_val, batch_size=64),
                              epochs = 30, validation_data = (y_train,y_val),
                              verbose = 2)

2022-06-28 05:27:06.300031: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
558/558 - 109s - loss: 0.5616 - accuracy: 0.8523 - val_loss: 0.1588 - val_accuracy: 0.9608
Epoch 2/30
558/558 - 107s - loss: 0.2910 - accuracy: 0.9351 - val_loss: 0.1788 - val_accuracy: 0.9557
Epoch 3/30
558/558 - 111s - loss: 0.2298 - accuracy: 0.9512 - val_loss: 0.1211 - val_accuracy: 0.9713
Epoch 4/30
558/558 - 106s - loss: 0.2112 - accuracy: 0.9564 - val_loss: 0.1956 - val_accuracy: 0.9546
Epoch 5/30
558/558 - 105s - loss: 0.1773 - accuracy: 0.9626 - val_loss: 0.0625 - val_accuracy: 0.9854
Epoch 6/30
558/558 - 105s - loss: 0.1594 - accuracy: 0.9661 - val_loss: 0.1173 - val_accuracy: 0.9708
Epoch 7/30
558/558 - 107s - loss: 0.1470 - accuracy: 0.9683 - val_loss: 0.0652 - val_accuracy: 0.9819
Epoch 8/30
558/558 - 105s - loss: 0.1314 - accuracy: 0.9714 - val_loss: 0.1245 - val_accuracy: 0.9611
Epoch 9/30
558/558 - 105s - loss: 0.1166 - accuracy: 0.9746 - val_loss: 0.0536 - val_accuracy: 0.9881
Epoch 10/30
558/558 - 105s - loss: 0.1151 - accuracy: 0.9752 - val_loss: 0.0526 - 

In [12]:
# inputs = keras.Input(shape=(28,28,1))
# # x = data_augmentation(inputs)
# x = layers.Conv2D(filters=64, kernel_size=3, activation = 'relu')(inputs)
# x = BatchNormalization()(x)
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.Conv2D(filters=128, kernel_size=3, activation = 'relu')(x)
# x = Dropout(rate = 0.25)(x)
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.Conv2D(filters=128, kernel_size=3, activation = 'relu')(x)
# x = BatchNormalization()(x)
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = Dropout(rate = 0.25)(x)


# x = layers.Flatten()(x)
# x = Dense(256, activation = 'relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(rate = 0.5)(x)

# outputs = layers.Dense(10, activation='softmax')(x)
# model = keras.Model(inputs=inputs, outputs=outputs)

# opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
# # opt = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [13]:
# fit = model.fit_generator(datagen.flow(x_train,x_val, batch_size=batch_size),
#                               epochs = 43, validation_data = (y_train,y_val),
#                               verbose = 2)

In [14]:
pred = model.predict(test)
pred_class = np.argmax(pred, axis=-1)
out = sample.copy()
out['Label'] = pred_class
out.to_csv("submission.csv", index=False)